# Nippon Telegraph and Telephone HOTEL

## Índice
1. [Proceso de ETL y Carga de Datos](#proceso-de-etl-y-carga-de-datos)
2. [Transformaciones en los Datos](#transformaciones-en-los-datos)
3. [Operaciones en PostgreSQL](#operaciones-en-postgresql)
4. [Reportes](#reportes)

In [1]:
import pandas as pd
import os

## Proceso de ETL y Carga de Datos

In [2]:
# Leer el archivo CSV
nombre_archivo = 'hotel_bookings.csv'

try:
  hotel_data = pd.read_csv(nombre_archivo, delimiter=",")
  #si es correcto imprimimos el dataframe
except FileNotFoundError:
  print("No existe el archivo '" + nombre_archivo + "'")
except pd.errors.ParserError:
  print("Verifica el formato csv del archivo'" + nombre_archivo + "'")
finally:
  print("============================================")
  print("Fin de la lectura del archivo"+nombre_archivo)
  print(" - Numero de columnas : "+str(len(hotel_data.columns)))
  print(" - Numero de registros : "+str(len(hotel_data)))
  print("============================================")


Fin de la lectura del archivohotel_bookings.csv
 - Numero de columnas : 31
 - Numero de registros : 119390


In [3]:
hotel_data.head(5)

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,booking_changes,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,3,NaN,NaN,0,Transient,0.0,0,0,Check-Out,1/07/15
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,4,NaN,NaN,0,Transient,0.0,0,0,Check-Out,1/07/15
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,0,NaN,NaN,0,Transient,75.0,0,0,Check-Out,2/07/15
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,0,304.0,NaN,0,Transient,75.0,0,0,Check-Out,2/07/15
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,0,240.0,NaN,0,Transient,98.0,0,1,Check-Out,3/07/15


## Transformaciones en los Datos <a name="data-transformation"></a>

#### Verificando los tipos de datos del df

In [4]:
# Verificación de los tipos de datos que está leyendo pandas
hotel_data.dtypes

hotel                              object
is_canceled                         int64
lead_time                           int64
arrival_date_year                   int64
arrival_date_month                 object
arrival_date_week_number            int64
arrival_date_day_of_month           int64
stays_in_weekend_nights             int64
stays_in_week_nights                int64
adults                              int64
children                          float64
babies                              int64
meal                               object
country                            object
market_segment                     object
distribution_channel               object
is_repeated_guest                   int64
previous_cancellations              int64
previous_bookings_not_canceled      int64
reserved_room_type                 object
assigned_room_type                 object
booking_changes                     int64
agent                             float64
company                           

#### Eliminando duplicados

In [5]:
# Visualización de duplicados
if hotel_data.duplicated().any():
    print("============================================")
    print("Numero de filas en el df original: "+str(len(hotel_data)))
    print("Exiten '"+str(len(hotel_data[hotel_data.duplicated()]))+"' filas duplicadas")
    hotel_data_unique = hotel_data.drop_duplicates() #Eliminacion de duplicados
    print("Quedan '"+ str(len(hotel_data_unique))+"' filas unicas")
    print("============================================")
else:
    print("No hay duplicados en el DataFrame.")

Numero de filas en el df original: 119390
Exiten '31999' filas duplicadas
Quedan '87391' filas unicas


#### Cambiando el tipo a fecha

In [6]:
#Cambiando la columna de fecha a tipo fecha
hotel_data_unique['reservation_status_date'] = pd.to_datetime(hotel_data_unique['reservation_status_date'], format='%d/%m/%y')

# convirtiendo al formato YYYYMMDD requerido
hotel_data_unique.loc[:,'reservation_status_date'] = hotel_data_unique['reservation_status_date'].dt.strftime('%Y/%m/%d')
hotel_data_unique #Visualizamos


C:\Users\josch\AppData\Local\Temp\ipykernel_560\336409435.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_data_unique['reservation_status_date'] = pd.to_datetime(hotel_data_unique['reservation_status_date'], format='%d/%m/%y')


,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,booking_changes,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,3,NaN,NaN,0,Transient,0.00,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,4,NaN,NaN,0,Transient,0.00,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,0,NaN,NaN,0,Transient,75.00,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,0,304.0,NaN,0,Transient,75.00,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,0,240.0,NaN,0,Transient,98.00,0,1,Check-Out,2015-07-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119385,City Hotel,0,23,2017,August,35,30,2,5,2,...,0,394.0,NaN,0,Transient,96.14,0,0,Check-Out,2017-09-06
119386,City Hotel,0,102,2017,August,35,31,2,5,3,...,0,9.0,NaN,0,Transient,225.43,0,2,Check-Out,2017-09-07
119387,City Hotel,0,34,2017,August,35,31,2,5,2,...,0,9.0,NaN,0,Transient,157.71,0,4,Check-Out,2017-09-07
119388,City Hotel,0,109,2017,August,35,31,2,5,2,...,0,89.0,NaN,0,Transient,104.40,0,0,Check-Out,2017-09-07


## Obtención de columnas a eliminar

In [7]:
#convertir las columnas a lista
todas_columnas_df = hotel_data_unique.columns.tolist()
#seleccionar columnas en numeros de la lista
columnas_a_eliminar = [2, 7, 8, 9, 10, 11, 12, 13,14, 15, 16, 17,18,19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]

# filtra los índices que están dentro del rango de la lista
columnas_a_eliminar_nombres = [todas_columnas_df[i] 
                               for i in columnas_a_eliminar 
                               if i < len(todas_columnas_df)
                              ]

# elimina las columnas
try:
    df = hotel_data_unique.drop(columns=columnas_a_eliminar_nombres)
except KeyError as e:
    print(f"Error: {e}")
df

,hotel,is_canceled,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,reservation_status_date
0,Resort Hotel,0,2015,July,27,1,2015-07-01
1,Resort Hotel,0,2015,July,27,1,2015-07-01
2,Resort Hotel,0,2015,July,27,1,2015-07-02
3,Resort Hotel,0,2015,July,27,1,2015-07-02
4,Resort Hotel,0,2015,July,27,1,2015-07-03
...,...,...,...,...,...,...,...
119385,City Hotel,0,2017,August,35,30,2017-09-06
119386,City Hotel,0,2017,August,35,31,2017-09-07
119387,City Hotel,0,2017,August,35,31,2017-09-07
119388,City Hotel,0,2017,August,35,31,2017-09-07


In [8]:
#Verificar si hay valores faltantes en las columnas seleccionadas como importantes
try:
    valores_faltantes = df.isnull()
    suma_valores_faltantes = valores_faltantes.sum()
    if suma_valores_faltantes.any():
        print("Hay valores nulos:")
        print("============================")
        print(suma_valores_faltantes)
    else:
        print("No hay valores nulos.")
        print("============================")
        print(suma_valores_faltantes)
except KeyError as e:
    print(f"Error: {e} -revisar DataFrame.")
except Exception as e:
    print(f"Error inesperado: {e}")

No hay valores nulos.
hotel                        0
is_canceled                  0
arrival_date_year            0
arrival_date_month           0
arrival_date_week_number     0
arrival_date_day_of_month    0
reservation_status_date      0
dtype: int64


In [9]:
#Verificar si hay valores ceros en las columnas seleccionadas como importantes
try:
    valores_ceros = (df == 0).any()
    if valores_ceros.any():
        print("Hay ceros en las columnas")
        print("============================")
        print(valores_ceros)
    else:
        print("No hay ceros en las columnas.")
        print("============================")
        print(valores_ceros)
except KeyError as e:
    print(f"Error: {e} - revisar DataFrame.")
except Exception as e:
    print(f"Error inesperado: {e}")
    
#Vemos que hay ceros en 'is_canceled' pero no hacemos caso ya que 0 si es un valor en la columna.

Hay ceros en las columnas
hotel                        False
is_canceled                   True
arrival_date_year            False
arrival_date_month           False
arrival_date_week_number     False
arrival_date_day_of_month    False
reservation_status_date      False
dtype: bool


## Operaciones en PostgreSQL

### Conexión

In [10]:
#Importamos pyscopg2
import psycopg2

#### Inicialización y creación de la Base de datos

In [17]:
# Parametros
host = 'localhost'
user = 'nttdata'
password = 'nttpwd'
database_name = 'hotel_bd'

try:
    connection = psycopg2.connect(
        host=host,
        user=user,
        password=password,
        database='postgres'
    )

    # se crea el cursos para la conexion
    cursor = connection.cursor()
    connection.autocommit = True

    # ejecutamos una consulta para crear la bd
    cursor.execute(f"CREATE DATABASE {database_name}")

    print(f"La base de datos '{database_name}' ha sido creada correctamente.")

except psycopg2.Error as e:
    print("Error al crear la base de datos:", e)

La base de datos 'hotel_bd' ha sido creada correctamente.


#### Creación la tabla hotel

In [18]:
sql_create_table = """
CREATE TABLE hotel (
    hotel VARCHAR,
    is_canceled INTEGER,
    arrival_date_year INTEGER,
    arrival_date_month VARCHAR,
    arrival_date_week_number INTEGER,
    arrival_date_day_of_month INTEGER,
    reservation_status_date DATE
);
"""
cursor.execute(sql_create_table)
connection.commit()
print("La tabla 'hotel' ha sido creada correctamente.")

La tabla 'hotel' ha sido creada correctamente.


#### Verificación de existencia de la tabla

In [19]:
cursor.execute("SELECT datname FROM pg_database;")
databases = cursor.fetchall()
print("bases de datos:")
print("========================")
for db in databases:
    print(db[0])

bases de datos:
postgres
template1
template0
odoo
ntt_db
hotel_bd


In [20]:
cursor.execute("SELECT table_name FROM information_schema.tables WHERE table_schema = 'public';")
tables = cursor.fetchall()
print("\nTablas en la bd:")
print("======================")
for table in tables:
    print(table[0])


Tablas en la bd:
hotel


#### Inserción del DataFrame en la base de datos

In [22]:
from sqlalchemy import create_engine
import pandas as pd
table_name = 'hotel'
engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}/{database_name}')
df.to_sql(table_name, con=engine, index=False, if_exists='replace')

engine.dispose()

print(f"Datos insertados en la tabla '{table_name}' correctamente.")

Datos insertados en la tabla 'hotel' correctamente.


<!--#### Visualizamos los datos ingresados en la base de datos  utilizando pandas-->
#### Visualización de la integridad de la data en la base de datos
<!-- * Consultas para probar  que los datos se hayan cargado correctamente en la bd-->

In [23]:
sql_query = "SELECT * FROM hotel"#Seleccionamos la tabla hotel.

# Cargamos los datos del df a la tabla creada
df_en_bd = pd.read_sql(sql_query, engine)
# Visualizamos el df desde la bd creada
df_en_bd


,hotel,is_canceled,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,reservation_status_date
0,Resort Hotel,0,2015,July,27,1,2015-07-01
1,Resort Hotel,0,2015,July,27,1,2015-07-01
2,Resort Hotel,0,2015,July,27,1,2015-07-02
3,Resort Hotel,0,2015,July,27,1,2015-07-02
4,Resort Hotel,0,2015,July,27,1,2015-07-03
...,...,...,...,...,...,...,...
87386,City Hotel,0,2017,August,35,30,2017-09-06
87387,City Hotel,0,2017,August,35,31,2017-09-07
87388,City Hotel,0,2017,August,35,31,2017-09-07
87389,City Hotel,0,2017,August,35,31,2017-09-07


In [24]:
#Cuantos hoteles diferentes hay
sql_query2 = """SELECT COUNT(DISTINCT hotel) AS hoteles_distintos FROM hotel;"""
df_en_bd2 = pd.read_sql(sql_query2, engine)
df_en_bd2

,hoteles_distintos
0,2


#### Cierre de la conexión con la base de datos

In [25]:
# cierra el cursor y la conexión
cursor.close()
connection.close()

## Reportes

### Consulta 1: Número de cancelaciones por año

In [26]:
#numero de cancelaciones por año
query_11 = """
SELECT EXTRACT(YEAR FROM reservation_status_date)::INTEGER as year, COUNT(*) as num_cancelaciones
FROM hotel
WHERE is_canceled = 1
GROUP BY year
ORDER BY year

"""
#Cancelaciones por año, detalladas por mes
query_12 =  """
SELECT EXTRACT(YEAR FROM reservation_status_date)::INTEGER as year,
       TO_CHAR(reservation_status_date, 'Month') as month,
       COUNT(*) as num_cancelaciones
FROM hotel
WHERE is_canceled = 1
GROUP BY year, month
ORDER BY year, MIN(EXTRACT(MONTH FROM reservation_status_date))
"""

df_query_11 = pd.read_sql(query_11, con=engine)
df_query_12 = pd.read_sql(query_12, con=engine)

# Mostrar los resultados en el notebook
print("Consulta 1 - Número de cancelaciones por año:")
display(df_query_11)
# Muestra los resultados en el cuaderno de Jupyter
print("Detalle por año:")
display(df_query_12)

Consulta 1 - Número de cancelaciones por año:


,year,num_cancelaciones
0,2014,12
1,2015,3226
2,2016,12779
3,2017,8003


Detalle por año:


,year,month,num_cancelaciones
0,2014,October,11
1,2014,November,1
2,2015,January,76
3,2015,February,25
4,2015,March,39
5,2015,April,89
6,2015,May,161
7,2015,June,329
8,2015,July,443
9,2015,August,402


### Consulta 2: ¿Cuál es la temporada más alta de cancelaciones (Mes)?

In [27]:
#Temporada (mes) con más cancelaciones
query_p21 = """
SELECT
  TO_CHAR(reservation_status_date, 'TMmonth') as month,
  COUNT(*) as num_cancelaciones
FROM hotel
WHERE is_canceled = 1
GROUP BY month
ORDER BY num_cancelaciones DESC
LIMIT 1
"""

query_p22 = """
SELECT
  TO_CHAR(reservation_status_date, 'month') as month,
  COUNT(*) as num_cancelaciones
FROM hotel
WHERE is_canceled = 1
GROUP BY month
ORDER BY num_cancelaciones DESC
"""

# Leer datos desde la base de datos
df_p21 = pd.read_sql(query_p21, con=engine)
df_p22 = pd.read_sql(query_p22, con=engine)

# Mostrar los resultados en el notebook
print("Consulta 2 - Temporada(mes) con más cancelaciones:")
display(df_p21)
print(" ")
print("Temporada(mes) por año y sus cancelaciones: ")
display(df_p22)

Consulta 2 - Temporada(mes) con más cancelaciones:


,month,num_cancelaciones
0,march,2652


 
Temporada(mes) por año y sus cancelaciones: 


,month,num_cancelaciones
0,march,2652
1,february,2561
2,january,2498
3,april,2338
4,may,2310
5,july,2097
6,june,1996
7,august,1704
8,october,1586
9,september,1499
